# Visualizing Field Coverage with a Stacked Graph

The following is an example of how to pull data from our backend to visualize it in this notebook using d3.  This example is not a particularly useful for actually analyzing field coverage, but it does nicely illustrate the flow from our graphql backend to this notebook to a visual using a python library or something like d3js.  Please see the resulting graph as a proof of concept, not a sharable visual.


## Import all Our necessary python dependencies



In [19]:
from gql import gql, Client # generate graphql queries and setup client for talking to graphql endpoints.
from gql.transport.requests import RequestsHTTPTransport # connect our client to a gql endpoint over https
import pandas as pd # for manipulating our data 
import json # parse returned gql into json for our d3 visualization
from IPython.display import display, Javascript, HTML # allow us to use javascript within python 
                                                      # and manipulate the dom of this notebook


## Import Our Data

For this visualization we want to have an x-axis displaying each podspec field, and a stacked barchart showing it's test  hits and e2e hits.  So our data should look, basically, like this:

In [20]:
sample_data = [
 { "podspec_field": "containers", "e2e_hits": "10", "conf_hits": "15"},
 { "podspec_field": "volumes", "e2e_hits": "12", "conf_hits": "18"},
 { "podspec_field": "nodes", "e2e_hits": "05", "conf_hits": "20"},
 { "podspec_field": "storage", "e2e_hits": "01", "conf_hits": "15"},
 { "podspec_field": "k8s", "e2e_hits": "02", "conf_hits": "10"},
]

We have a view in our db called 'podspec_field_report' that contains each of these values, so we can setup a graphql query that asks _just_ for them.  We'll set it to a variable to make it easier to execute and/or change later.

In [21]:
podspec_field_report_query=gql("""
query{
  podspec_field_report {
    podspec_field
    e2e_hits
    conf_hits
  }
}
""")

Our backend is reachable through a single api endpoint, located here:

In [22]:
BACKEND_URL = "https://zz-hasura.sharing.io/v1/graphql"

To be able to post this query to our graphql backend, we need to setup a gql client that fetches from this endpoint.  the python graphql client calls this the `transport`

In [23]:
_transport = RequestsHTTPTransport(
    url=BACKEND_URL,
    use_json=True
)

client = Client(
    transport=_transport,
    fetch_schema_from_transport=True
)

We can now use the client to execute this query.  Printing it out shows that our returned data matches our desired structure.

In [24]:
podspec_field_report = client.execute(podspec_field_report_query)["podspec_field_report"]
print(json.dumps(podspec_field_report, indent=4, sort_keys=True))

[
    {
        "conf_hits": 0,
        "e2e_hits": 0,
        "podspec_field": "ephemeralContainers"
    },
    {
        "conf_hits": 0,
        "e2e_hits": 0,
        "podspec_field": "overhead"
    },
    {
        "conf_hits": 0,
        "e2e_hits": 0,
        "podspec_field": "preemptionPolicy"
    },
    {
        "conf_hits": 0,
        "e2e_hits": 0,
        "podspec_field": "shareProcessNamespace"
    },
    {
        "conf_hits": 0,
        "e2e_hits": 0,
        "podspec_field": "topologySpreadConstraints"
    },
    {
        "conf_hits": 0,
        "e2e_hits": 8,
        "podspec_field": "readinessGates"
    },
    {
        "conf_hits": 0,
        "e2e_hits": 16,
        "podspec_field": "dnsConfig"
    },
    {
        "conf_hits": 0,
        "e2e_hits": 16,
        "podspec_field": "hostIPC"
    },
    {
        "conf_hits": 0,
        "e2e_hits": 16,
        "podspec_field": "hostPID"
    },
    {
        "conf_hits": 0,
        "e2e_hits": 32,
        "podspec_field"

Now we can visualize it!

## Visualizing the Data

I am using the example provided by Stefan Liipens' excellent blogpost: [Custom D3 Visualization in a Jupyter Notebook](https://www.stefaanlippens.net/jupyter-custom-d3-visualization.html)

First, we tell our notebook to require in d3 (via it's minified file) and our  barchart visualization held in this local notebook folder.

In [25]:
display(Javascript("require.config({paths: {d3: 'https://d3js.org/d3.v3.min'}});"))
display(Javascript(filename="barchart.js"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The barchart visualization is expecting a container and data.  The container is the dom element we'll append our svg to, and then our imported data will generate a bar chart.

element is a global variable indicating the output cell for wherever the code is located, and so by wrapping our javascript in closures, we can keep element local to any particular cell.

So here we are defining a python function, that is just a string interpoloation of a javascript function, that thinks it is latching to an html page, but is actually latching onto a single cell (within an html page).

In [26]:
def draw_barchart():
    display(Javascript("""
        (function(element){
            require(['barchart'], function(barchart) {
                barchart(element.get(0), %s);
            });
        })(element);
    """ % json.dumps(data)))

To include the visualization in our notebook, we just call the function.

In [27]:
draw_barchart()

<IPython.core.display.Javascript object>

Terrible graphic but successful example!